In [ ]:
# option 1 
# use all available data for training and testing on same dataset
# e.g., 100 math questions prepared, and a few questions to be tested
# option 2
# 70/100 for training and 30/100 for testing 

# option 3
# K-fold cross vaildation
# 1000 samples 20/100 for testing and 80/100 for training x 5 sets
# take average

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
digits = load_digits()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(digits.data,digits.target,test_size=0.3)

In [3]:
# every time you reexectue it, the score changed. You cannot only run more time since the 
# dataset split everytime 

In [4]:
# Logistic Regression
lr = LogisticRegression(solver='liblinear',multi_class='ovr')
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

"""

solver : {'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'},             default='lbfgs'

        - For small datasets, 'liblinear' is a good choice, whereas 'sag'
          and 'saga' are faster for large ones;
        - For multiclass problems, only 'newton-cg', 'sag', 'saga' and
          'lbfgs' handle multinomial loss;
        - 'liblinear' is limited to one-versus-rest schemes.

    .. warning::
       The choice of the algorithm depends on the penalty chosen:
       Supported penalties by solver:

       - 'newton-cg'   -   ['l2', 'none']
       - 'lbfgs'       -   ['l2', 'none']
       - 'liblinear'   -   ['l1', 'l2']
       - 'sag'         -   ['l2', 'none']
       - 'saga'        -   ['elasticnet', 'l1', 'l2', 'none']

multi_class : {'auto', 'ovr', 'multinomial'}, default='auto'
    If the option chosen is 'ovr', then a binary problem is fit for each
    label.
    
"""

"\n\nsolver : {'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'},             default='lbfgs'\n\n        - For small datasets, 'liblinear' is a good choice, whereas 'sag'\n          and 'saga' are faster for large ones;\n        - For multiclass problems, only 'newton-cg', 'sag', 'saga' and\n          'lbfgs' handle multinomial loss;\n        - 'liblinear' is limited to one-versus-rest schemes.\n\n    .. warning::\n       The choice of the algorithm depends on the penalty chosen:\n       Supported penalties by solver:\n\n       - 'newton-cg'   -   ['l2', 'none']\n       - 'lbfgs'       -   ['l2', 'none']\n       - 'liblinear'   -   ['l1', 'l2']\n       - 'sag'         -   ['l2', 'none']\n       - 'saga'        -   ['elasticnet', 'l1', 'l2', 'none']\n\nmulti_class : {'auto', 'ovr', 'multinomial'}, default='auto'\n    If the option chosen is 'ovr', then a binary problem is fit for each\n    label.\n    \n"

In [5]:
# SVM

svm = SVC(gamma='auto')
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.5092592592592593

In [6]:
# Random Forest

rf = RandomForestClassifier(n_estimators=40)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.975925925925926

# KFold cross validation
- Basic example

In [7]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3) # 3 sets
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [8]:
for train_index, test_index in kf.split([1,2,3,4,5,6,7,8,9]):
    print(train_index, test_index)

[3 4 5 6 7 8] [0 1 2]
[0 1 2 6 7 8] [3 4 5]
[0 1 2 3 4 5] [6 7 8]


# Use KFold for our digits example
- just for explanation 

In [9]:
def get_score(model, X_train, X_test, y_train, y_test): # take model first..
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [10]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=3)

scores_logistic = []
scores_svm = []
scores_rf = []

for train_index, test_index in folds.split(digits.data,digits.target):
    X_train, X_test, y_train, y_test = digits.data[train_index], digits.data[test_index], \
                                       digits.target[train_index], digits.target[test_index]
    scores_logistic.append(get_score(LogisticRegression(solver='liblinear',multi_class='ovr'), X_train, X_test, y_train, y_test))  
    scores_svm.append(get_score(SVC(gamma='auto'), X_train, X_test, y_train, y_test))
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=40), X_train, X_test, y_train, y_test))

In [11]:
scores_logistic

[0.8948247078464107, 0.9532554257095158, 0.9098497495826378]

In [12]:
scores_svm

[0.3806343906510851, 0.41068447412353926, 0.5125208681135225]

In [13]:
scores_rf

[0.9298831385642737, 0.9515859766277128, 0.9198664440734557]

# cross_val_score function
- we use this in reality 

In [14]:
from sklearn.model_selection import cross_val_score

# cross_val_score uses stratifield kfold by default

In [15]:
# Logistic regression model performance using cross_val_score
cross_val_score(LogisticRegression(solver='liblinear',multi_class='ovr'), digits.data, digits.target,cv=3)

array([0.89482471, 0.95325543, 0.90984975])

In [16]:
# svm model performance using cross_val_score
cross_val_score(SVC(gamma='auto'), digits.data, digits.target,cv=3)

array([0.38063439, 0.41068447, 0.51252087])

In [17]:
# random forest performance using cross_val_score
cross_val_score(RandomForestClassifier(n_estimators=40),digits.data, digits.target,cv=3)

array([0.94490818, 0.95659432, 0.92320534])

# Parameter tunning using k fold cross validation

In [18]:
scores1 = cross_val_score(RandomForestClassifier(n_estimators=5),digits.data, digits.target, cv=10)
np.average(scores1)

0.8664525139664805

In [19]:
scores2 = cross_val_score(RandomForestClassifier(n_estimators=20),digits.data, digits.target, cv=10)
np.average(scores2)

0.9393420235878336

In [20]:
scores3 = cross_val_score(RandomForestClassifier(n_estimators=30),digits.data, digits.target, cv=10)
np.average(scores3)

0.941548727498448

In [21]:
scores4 = cross_val_score(RandomForestClassifier(n_estimators=40),digits.data, digits.target, cv=10)
np.average(scores4)

0.9459993792675357

In [27]:
# Here we used cross_val_score to fine tune our random forest classifier and 
# figured that having around 40 trees in random forest gives best result.

n_estimators=[5,20,30,40]
average_scores ={}
for ne in n_estimators:
    cv_scores = cross_val_score(RandomForestClassifier(n_estimators=ne),digits.data, digits.target, cv=10)
    average_scores['the average score of' +'_'+str(ne)+'_'+'is'] = np.average(cv_scores)
average_scores

{'the average score of_5_is': 0.8870360024829299,
 'the average score of_20_is': 0.9348758535071383,
 'the average score of_30_is': 0.9393420235878335,
 'the average score of_40_is': 0.9471229050279328}